## Алгоритм компенсации

Обозначим список ключевых изменяющихся параметров (напрямую влияющие на фактический вылив):

1. Ограничения насосов (мин/макс скорость, и их параметры ускорения/остановки)
2. Норма внесения
3. Максимально допустимая скорость дрона

Обозначим порядок в котором будем менять параметры:
1. Скорость дрона
2. Норма внесения


Возьмём в качестве критического значения вызывающего пересчёт с компенсацией:

`max_available_error_litters = 1`

Также не маловажно определить политику важности направления ошибки (есть ли разница, и что важнее - перелить или недолить):
на текущий момент будем считать это равным

In [1]:
from policy import PolicyFactory, ViolationMode

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [49]:
policy = PolicyFactory.with_tolerance(over_tol=0.5,  # Допускаем переливать 0,5 л
                                      under_tol=0.5,  # Допускаем не доливать 0,5 л
                                      w_over=2,  # Переливание в 2 раза хуже
                                      violation=ViolationMode.ERROR  # Кидаем исключения при нарушениях
                                      )

target = 0
x = np.linspace(-10, 10, 21) / 10
y = [policy.cost(target, i) for i in x]
plt.plot(x, y)

ConstraintViolation: Нарушено ограничение политики

In [45]:
x[np.argmin(y)]

np.float64(0.0)